# Lab 12: Classification

This assignment covers **Chapter 17** from the textbook as well as lecture material from Weeks 12-13. Please complete this assignment by providing answers in cells after the question. Use **Code** cells to write and run any code you need to answer the question and **Markdown** cells to write out answers in words. After you are finished with the assignment, remember to download it as an **HTML file** and submit it in **ELMS**.

This assignment is due by **11:59pm on Thursday, May 5**. 

In [ ]:
import numpy as np
from datascience import *

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

Using the classifiers in `sklearn` is very similar to finding the least squares regression line using `LinearRegression`. We first create the model object, then fit the model with our data. In our case, we will fit the data using our train data, and then predict for the test data (using the `predict_proba` method). 

This assignment will cover the main steps for applying machine learning models.
- Create train and test sets
- Fit the models using train set
- Predict using the test set
- Evaluate models using metrics such as precision and recall
- Make your conclusions

## Brazilian Sign Language

Brazilian Sign Language is a visual language used primarily by Brazilians who are deaf.  It is more commonly called Libras.  People who communicate with visual language are called *signers*. 

Programs like Siri or Google Now begin the process of understanding human speech by classifying short clips of raw sound into basic categories called *phones*.  For example, the recorded sound of someone saying the word "robot" might be broken down into several phones: "rrr", "oh", "buh", "aah", and "tuh".  Phones are then grouped together into further categories like words ("robot") and sentences ("I, for one, welcome our new robot overlords") that carry more meaning.

A visual language like Libras has an analogous structure.  Instead of phones, each word is made up of several *hand movements*.  As a first step in interpreting Libras, we can break down a video clip into small segments, each containing a single hand movement.  The task is then to figure out what hand movement each segment represents.

We can do that with classification!

The [data](https://archive.ics.uci.edu/ml/machine-learning-databases/libras/movement_libras.names) in this exercise come from Dias, Peres, and Biscaro, researchers at the University of Sao Paulo in Brazil.  They identified 15 distinct hand movements in Libras (probably an oversimplification, but a useful one) and captured short videos of signers making those hand movements.  (You can read more about their work [here](http://ieeexplore.ieee.org/Xplore/login.jsp?url=http%3A%2F%2Fieeexplore.ieee.org%2Fiel5%2F5161636%2F5178557%2F05178917.pdf&authDecision=-203). The paper is gated, so you will need to use the UMD Wi-Fi or VPN to access it.)

For each video, they chose 45 still frames from the video and identified the location (in horizontal and vertical coordinates) of the signer's hand in each frame.  Since there are two coordinates for each frame, this gives us a total of 90 numbers summarizing how a hand moved in each video.  Those 90 numbers will be our *attributes*.

Each video is *labeled* with the kind of hand movement the signer was making in it.  Each label is one of 15 strings like "horizontal swing" or "vertical zigzag".

For simplicity, we're going to focus on distinguishing between just two kinds of movements: "horizontal straight-line" and "vertical straight-line".  We took the Sao Paulo researchers' original dataset, which was quite small, and used some simple techniques to create a much larger synthetic dataset.

These data are in the file `movements.csv`.  Run the next cell to load it.

In [ ]:
movements = Table.read_table("movements.csv")
movements.show(5)

### Step 1: Create train and test sets

First, let's split up the data into train and test sets. For this assignment, we will do a simple holdout set, assigning a random 20% of the data as the test data, and building the model on the remaining 80% of the data. 

<font color = 'red'>**Question 1. Create two Tables, one called `test` and one called `train`. The `test` table should contain a random 20% of the data, while the `train` Table should contain the other 80%.**<\font>

*Hint:* You can shuffle the entire dataset (sample the whole dataset without replacement), then just take the top 80% as your train data.

### Step 2: Fitting the models

Let's fit three different models: Logistic Regression, Decision Trees, and K-Nearest Neighbors. The model objects are initialized using the following code.

In [ ]:
# Create the model objects
logit = LogisticRegression()
tree = DecisionTreeClassifier()
knn = KNeighborsClassifier(n_neighbors = 7)

Note that we are using the default values for many of these. We generally would want to try a variety of different models with different parameters (for example, using different values for the stopping criteria in decision trees). We'll stick with just the defaults for now for this assignments. You can now use the `.fit` method to give it the data. 

<font color = 'red'>**Question 2. Using the `train` Table you created above, fit each of the three models. Use all of the Frame x and y variables as your features (predictors) to predict the Movement type.**</font>

If you're not sure about the exact format of the data needed, remember that you need to use `.rows` for the `X` values and `.column` for the `y` values. See lecture material and Labs 9 and 10 for how we fit linear regression models if you're still unsure of how to proceed.

You shouldn't need to change the outcome variable (`Movement type`) to a 0/1 variable. That is, you can use the data as it is without converting any of the categorical variables.

After you fit the models, you can use the `.classes_` instance variable to check the classes that are being predicted.

In [ ]:
knn.classes_

### Step 3: Predict Test Set

Just as with linear regression, we can use the `.predict_proba` method with the model objects. The `.predict_proba` method provides scores for each class. We need to select one of the two classes as the "positive" outcome. Since there isn't one outcome we're particularly interested in out of the two possibilities, we will just choose "horizontal straight-line" to be positive. 

> In other situations, there might be one that you care more about. For example, if you are trying to predict recidivism in order to provide social programs to help released prisoners at higher risk of returning, then your "positive" outcome -- the outcome you are trying to predict -- would be "return", so that is what you would want to set as `True` below. 

We can't simply evaluate using the scores, though. We need to set a threshold so that we predict that an observation is "horizontal straight-line" or not. Here, we will set the threshold to be 0.5, but remember, there is nothing special about this number. We could have easily selected any other number between 0 and 1, and we should, in practice, try a wide range of values and see how our models perform with each of these thresholds.

An example using K-Nearest Neighbors is shown below.

In [ ]:
# Setting a threshold (above means predicted to be horizontal straight-line)
threshold = 0.5

In [ ]:
# Make sure you fit the model before running this!
test_features = test.select(np.arange(90)).rows
knn_predicted = knn.predict_proba(test_features)[:,0] > threshold

Note that we are only taking the first value from each row, as evidenced by the `[:,0]`. This is because we want the predicted scores for horizontal straight-line, and that is the first value (see the `.classes_` instance variable above). The scores for each of the categories are given, but we only really care about one, because that's all we need since the scores will add up to 1 across each row. 

<font color = 'red'>**Question 3. Create an array called `expected` that contain the expected values based on the test set. The `expected` array should contain `True` if the observation is actually a "horizontal straight-line" and `False` otherwise. Note that the `expected` array is based entirely on the real dataset, and not on any predictions! Create additional arrays that contain the predicted values for each of the models that we've fit (call them `logit_predicted` and `tree_predicted`).**</font>

### Step 4: Evaluate

You can get a confusion matrix using the `confusion_matrix` function that we brought at the beginning. This is part of the `sklearn.metrics` module.

In [ ]:
conf_matrix = confusion_matrix(expected,knn_predicted)

In [ ]:
conf_matrix

The columns represent predictions and the rows represent actual values, so the top left is true negatives, the bottom right is true positives, the top right is false positives, and the bottom left is false negatives.

#### Evaluation metrics

Two metrics that are often more relevant than overall accuracy are **precision** and **recall**. 

Precision measures the accuracy of the classifier when it predicts an example to be positive. It is the ratio of correctly predicted positive examples to examples predicted to be positive. 

$$ Precision = \frac{TP}{TP+FP}$$

Recall measures the accuracy of the classifier to find positive examples in the data. 

$$ Recall = \frac{TP}{TP+FN} $$

By selecting different thresholds we can vary and tune the precision and recall of a given classifier. A conservative classifier (threshold 0.99) will classify a case as 1 only when it is *very sure*, leading to high precision. On the other end of the spectrum, a low threshold (e.g. 0.01) will lead to higher recall. 

We can use the `precision_score` and `recall_score` functions to find the value of these measures.

In [ ]:
precision_score(expected,knn_predicted)

In [ ]:
recall_score(expected,knn_predicted)

<font color = 'red'>**Question 4. Find the confusion matrix for the Logistic Regression and Decision Tree models. Using the confusion matrix, compute the accuracy, precision, and recall. Afterwards, use the `precision_score` and `recall_score` functions to verify your answer.**</font>

### Step 5: Repeating the steps

We've done one iteration ... but we've only done it with one threshold, and we haven't tuned the parameters much. We won't go through all of the various ways we can fine-tune our models, but we can show how it is done: using loops.

<font color = 'red'>**Question 5. Write a loop that tries thresholds of .1, .3, .5, .7, and .9. Store the precision of each model at each threshold within their own arrays, named `knn_precision`, `logit_precision` and `tree_precision`. Do the same for recall (replacing precision with recall in the naming convention).**</font>

The loop has been started below for you.

In [ ]:
knn_precision = make_array()
logit_precision = make_array()
tree_precision = make_array()
    
knn_recall = make_array()
logit_recall = make_array()
tree_recall = make_array()

# Set up models and fit them here
...

for i in make_array(.1,.3,.5,.7,.9):
    ...
    knn_precision = np.append(knn_precision, ...)
    logit_precision = np.append(logit_precision, ...)
    tree_precision = np.append(tree_precision, ...)
    
    knn_recall = np.append(knn_recall, ...)
    logit_recall = np.append(logit_recall, ...)
    tree_recall = np.append(tree_recall, ...)
    

In [ ]:
# You can use these to look at results
precision_results = Table().with_columns('Threshold', make_array(.1, .3, .5, .7, .9),
                              'KNN Precision', knn_precision,
                              'Logistic Regression Precision', logit_precision,
                              'Decision Tree Precision', tree_precision)

recall_results = Table().with_columns('Threshold', make_array(.1, .3, .5, .7, .9),
                              'KNN Recall', knn_recall,
                              'Logistic Regression Recall', logit_recall,
                              'Decision Tree Recall', tree_recall)

In [ ]:
precision_results.show(3)

In [ ]:
recall_results.show(3)

<font color = 'red'>**Question 6. What model and threshold combination gives the best precision? The best recall? If there are ties, you can just choose one.**</font>

*Hint:* You can use `.sort` to sort a Table by one variable.

#### Step 6: Model Selection and Conclusions

Generally, when deciding on the best model, we compare the models we fit with each other, as well as against a baseline.

<font color = 'red'>**Question 7. Consider the threshold used in the best model by precision chosen in Question 6. If we were to try to predict whether a hand movement was "horizontal straight-line" or not completely randomly, how often would we be right? That is, what would be our precision if we were guessing completely randomly? How well does our best model perform compared to that baseline?**</font>